In [ ]:
def count_unique_words(corpus):
    words_freq = dict()
    for sentence in tqdm(corpus):
        # print(sentence)
        for word in str(sentence).split():
            if word not in words_freq:
                words_freq[word] = 1
            else:
                words_freq[word] += 1
    return words_freq
    
words_freq = count_unique_words(df["queryText"])
len(words_freq)

In [2]:
import pandas as pd
from tqdm.auto import tqdm
import random
from transformers import AutoTokenizer

class DatasetHandler:
    def __init__(self, file_address):
        tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
        self.df = pd.read_csv(file_address) if "csv" in file_address else pd.read_pickle(file_address)
        self.df["similar"] = 1
        self.package_to_id, self.id_to_package = self.label_packages(self.df["packageName"], start_id=len(tokenizer.get_vocab()))
        
    def label_packages(self, packages: list, start_id=0):
        package_to_id = dict()
        id_to_package = dict()
        id_counter = start_id
        for package in tqdm(packages, desc="Labeling Packages"):
            if package not in package_to_id:
                package_to_id[package] = id_counter
                id_to_package[id_counter] = package
                id_counter += 1
        return package_to_id, id_to_package
    
    def save_negative_sampled_dataset(self, neg_to_pos_ratio:int=4, output_name="neg_sampled_data.pkl", seed=0):
        random.seed(seed)
        negative_rows = []
        random_choices = list(self.package_to_id.keys())
        for query_text, package_name in tqdm(zip(self.df["queryText"], self.df["packageName"]), total=len(self.df["queryText"])):
            for _ in range(neg_to_pos_ratio):
                random_package = random.choice(random_choices)
                while random_package == package_name:
                    random_package = random.choice(random_choices)
                negative_rows.append({'queryText': query_text, "packageName": random_package, "similar": 0})
        neg_df = pd.DataFrame(negative_rows)
        new_df = pd.concat([self.df, neg_df], ignore_index=True)
        new_df.to_pickle("data/neg_sampled_data.pkl")
        return new_df

# dataset_handler = DatasetHandler("data/SA_history_clicks_of_98.csv")
dataset_handler = DatasetHandler("../data/neg_sampled_data5.pkl")

In [ ]:
# new_df = dataset_handler.save_negative_sampled_dataset(neg_to_pos_ratio=5)
# new_df

In [3]:
dataset_handler.df

,queryText,packageName,similar
0,توشمال,com.example.masood.yocheneapp,1
1,rush,com.ketchapp.rush,1
2,شب چله,yalda.ir.kr.si.ma.usn,1
3,عشق,com.roman.jadogareeshgh,1
4,کلابی,ir.mmdali.cluby,1
...,...,...,...
65689147,کاهتهتخنتالفل 644 علیر,com.OmidStudio.Monster17,1
65689148,کاهتهتخنتالفل 644 علیر,com.zeinabbanoo16.pareshehaiajani,1
65689149,کاهتهتخنتالفل 644 علیر,app.kalagsm.com,1
65689150,کاهتهتخنتالفل 644 علیر,ir.hassanmansouri.mohsenyegane,1


In [13]:
import torch
from src.SpanPooling import *

class RepresentationBuilder(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim=300):
        super(RepresentationBuilder, self).__init__()
        self.embedding = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.pooling_module = get_pooling_module("avg")
        
    def forward(self, input_ids, attention_mask):
        out = self.embedding(input_ids)
        print(out)
        out = self.pooling_module(out, attention_mask)
        return out

def 

In [68]:
import torch

class Siamese(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Siamese, self).__init__()
        self.representation_builder = torch.nn.Sequential(
            RepresentationBuilder(vocab_size, embedding_dim),
            torch.nn.ReLU()
        )
        self.loss = torch.nn.CosineEmbeddingLoss(margin=0)
        
    def forward(self, x1, x2):
        x1_repr = self.representation_builder(x1)
        x2_repr = self.representation_builder(x2)
        print(x1_repr)
        print(x2_repr)
        print(self.loss(x1_repr, x2_repr, target=torch.tensor([-1, 1])))
        

In [71]:
import torch

class Trainer:
    def __init__(self, vocab_size, embedding_dim):
        self.siamese = Siamese(vocab_size, embedding_dim)
    
    def train(self):
        self.siamese(torch.tensor([[1], [0, 0, 0, 0]]), torch.tensor([[1, 2, 3, 3], [9, 9, 9, 9]]))

In [72]:
trainer = Trainer(vocab_size=10, embedding_dim=5)
trainer.train()



ValueError: expected sequence of length 1 at dim 1 (got 4)